<a href="https://colab.research.google.com/github/jaymonty/NLP_News_App/blob/main/Flan_T5_QLoRA_Eval(GPU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct  2 22:56:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# install Hugging Face Libraries
!pip install "peft==0.2.0"
!pip install "transformers==4.27.1" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib --upgrade --quiet
# install additional dependencies needed for training
!pip install rouge-score tensorboard py7zr
# ! pip install datasets transformers rouge-score nltk
! pip install datasets rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# import torch
# device = torch.device("cpu")

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "emonty777/QLoRA-Flan-T5-Small"

config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer for GPU
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_sa

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [ ]:
# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")

Peft model loaded


In [ ]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub and get a sample
dataset = load_dataset('cnn_dailymail','3.0.0')
sample = dataset['test'][randrange(len(dataset["test"]))]

  0%|          | 0/3 [00:00<?, ?it/s]

input sentence: (CNN)A former U.S. Army enlistee who posted on Facebook about "the adrenaline rush" of dying in jihad was arrested Friday and charged with trying to detonate a car bomb at Fort Riley military base in Kansas, authorities said. A second man, who allegedly knew about the bomb plot but didn't call authorities, was charged with failing to report a felony. John T. Booker Jr. of Topeka, an American citizen also known as Mohammed Abdullah Hassan, was taken into custody near Manhattan, Kansas, in a van that contained what he thought was a bomb, the criminal complaint said. The "bomb" had actually been put together by two confidential informants with nonexplosive materials, the complaint said. Fort Riley's security was never breached and no people were in danger, the U.S. Justice Department said in a press release. Booker enlisted in the Army last year and was due to ship out to basic training April 7, 2014, said Army spokesman Wayne Hall. The criminal complaint said the FBI ques

In [ ]:
input_ids = tokenizer(sample["article"], return_tensors="pt", truncation=True).input_ids
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=120, do_sample=False, top_p=0.9)
print(f"input sentence: {sample['article']}\n{'---'* 20}")

print(f"summary:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

input sentence: (CNN)A former U.S. Army enlistee who posted on Facebook about "the adrenaline rush" of dying in jihad was arrested Friday and charged with trying to detonate a car bomb at Fort Riley military base in Kansas, authorities said. A second man, who allegedly knew about the bomb plot but didn't call authorities, was charged with failing to report a felony. John T. Booker Jr. of Topeka, an American citizen also known as Mohammed Abdullah Hassan, was taken into custody near Manhattan, Kansas, in a van that contained what he thought was a bomb, the criminal complaint said. The "bomb" had actually been put together by two confidential informants with nonexplosive materials, the complaint said. Fort Riley's security was never breached and no people were in danger, the U.S. Justice Department said in a press release. Booker enlisted in the Army last year and was due to ship out to basic training April 7, 2014, said Army spokesman Wayne Hall. The criminal complaint said the FBI ques

### Eval

In [ ]:
!pip install rouge

In [ ]:
# Import necessary libraries
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset
from rouge import Rouge

In [ ]:
# Load only the first 10 samples of the test dataset  <---- just for sample of 10
#dataset = load_dataset('cnn_dailymail', '3.0.0', split='test[:10]')

# Evaluate on full test set
dataset = load_dataset('cnn_dailymail', '3.0.0', split='test')

In [ ]:
!nvidia-smi
import torch
torch.cuda.is_available()

Mon Oct  2 23:03:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    52W / 400W |   2085MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

True

In [ ]:
def generate_summary(batch):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Determine the device to use
    model.to(device)  # Move the model to the GPU
    # Ensure padding and truncation
    inputs = tokenizer(batch["article"], return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)  # Move the inputs to the GPU
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=120,
        do_sample=False,
        #top_p=0.9
    )
    # Decode all generated summaries
    summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs.cpu()]  # Move outputs back to CPU for decoding
    return {"summary": summaries}


In [ ]:
# Process the dataset with progress bar & increase batch_size for processing full test dataset
summary_dataset = dataset.map(generate_summary, batched=True, batch_size=16)

# Extract generated and reference summaries
generated_summaries = summary_dataset["summary"]
reference_summaries = [item["highlights"] for item in dataset]

# Compute ROUGE scores
rouge = Rouge()
scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Output the scores
print(scores)

  0%|          | 0/719 [00:00<?, ?ba/s]

{'rouge-1': {'r': 0.34581471693347327, 'p': 0.38095507048313104, 'f': 0.3482476941930601}, 'rouge-2': {'r': 0.14615871503938022, 'p': 0.15544386062538168, 'f': 0.14347413417736696}, 'rouge-l': {'r': 0.32564770330481585, 'p': 0.35891668231561674, 'f': 0.32796570393137886}}


In [ ]:
from re import VERBOSE

def generate_summary(batch):
    # Ensure padding and truncation
    inputs = tokenizer(batch["article"], return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=120,
        do_sample=False,
        #top_p=0.9
    )
    # Decode all generated summaries
    summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return {"summary": summaries}

# Process the dataset with progress bar & increase batch_size for processing full test dataset
summary_dataset = dataset.map(generate_summary, batched=True, batch_size=16)


# Extract generated and reference summaries
generated_summaries = summary_dataset["summary"]
reference_summaries = [item["highlights"] for item in dataset]


# Compute ROUGE scores
rouge = Rouge()
scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Output the scores
print(scores)

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


{'rouge-1': {'r': 0.3555236214059743, 'p': 0.3312556987131762, 'f': 0.3403180360080413}, 'rouge-2': {'r': 0.13893932354907962, 'p': 0.13047992747992748, 'f': 0.1330749001151398}, 'rouge-l': {'r': 0.3245258673493968, 'p': 0.30492665919334727, 'f': 0.3120168974422899}}


{'rouge-1': {'r': 0.3393922302745832, 'p': 0.31269788895214956, 'f': 0.3233639945519167},

'rouge-2': {'r': 0.12221458487312147, 'p': 0.11488208616780046, 'f': 0.11733183925696884},

'rouge-l': {'r': 0.32062098844451786, 'p': 0.29718615712462826, 'f': 0.30647204500867536}}

In [ ]:
import pandas as pd

# Transforming the scores into a DataFrame
df_scores = pd.DataFrame(scores).T

# Naming the columns for better understanding
df_scores.columns = ['Recall', 'Precision', 'F1 Score']

# Display the DataFrame
print(df_scores)

           Recall  Precision  F1 Score
rouge-1  0.339392   0.312698  0.323364
rouge-2  0.122215   0.114882  0.117332
rouge-l  0.320621   0.297186  0.306472


### Evaluate Model on CNN Dailymail Test Data w/ Rouge Metric

In [ ]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")


def evaluate_peft_model(sample, max_target_length=120):
    # generate summary
    input_ids = tokenizer(sample["article"], return_tensors="pt", truncation=True).input_ids
    outputs = model.generate(input_ids=input_ids, do_sample=False, top_p=0.9, max_new_tokens=max_target_length)
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)

    # decode eval sample
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(sample['highlights'] != -100, sample['highlights'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    return prediction, labels

# load test dataset from distk
test_dataset = sample

# run predictions
# this can take ~45 minutes
predictions, references = [] , []
for sample in tqdm(test_dataset):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)

# compute metric
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)

# print results
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")

  0%|          | 0/1 [00:00<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 29>:30                                                                            │
│ in evaluate_peft_model:11                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: string indices must be integers